In [ ]:
import pandas as pd
import numpy as np
import holidays
import random
from datetime import datetime, timedelta

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from keras.callbacks import EarlyStopping
import numpy as np
from keras.models import load_model

Preparacion de los datos

In [4]:
#load data
df=pd.read_csv("C:\\Users\\MSI SWORD 15 A12V\\Downloads\\Datos de vuelo\\Prediccion de destinos populares\\T_DB1B_MARKET_20250607_092242\\T_DB1B_MARKET.csv")

In [ ]:


# Suponiendo que df es tu DataFrame original
df_data_import = df[['YEAR', 'QUARTER', 'ORIGIN', 'DEST','PASSENGERS']].copy()
# df_data_import.dtypes``

# 1. Generar fechas aleatorias por trimestre (no vectorizado)
def random_date_in_quarter(year, quarter):
    quarter_start=pd.Timestamp(year=year, month=(quarter-1)*3+1, day=1)
    quarter_end=quarter_start+pd.offsets.QuarterEnd()
    # print(quarter_start, quarter_end)
    random_days=np.random.randint(0, (quarter_end - quarter_start).days+1)
    random_date = quarter_start + pd.Timedelta(days=random_days)
    # print(random_days,quarter_start+pd.Timedelta(days=random_days))
    return random_date

#2. Aplicar a todo el DataFrame
df_data_import['DATE'] = df_data_import.apply(
    lambda row: random_date_in_quarter(int(row['YEAR']), int(row['QUARTER'])),
    axis=1
)
us_holidays = holidays.US(years=df_data_import['YEAR'].unique().tolist())

df_data_import['HOLIDAY']=df_data_import['DATE'].apply(lambda x: x in us_holidays).astype(int)
df_data_import['VACATION']=df_data_import['DATE'].dt.month.isin([7, 12]).astype(int)
df_data_import['IMPORTANT_EVENT']=df_data_import['DATE'].dt.day.isin([1, 15]).astype(int)


# Resultado
print(df_data_import.head(50))

    YEAR  QUARTER ORIGIN DEST  PASSENGERS       DATE  HOLIDAY  VACATION  \
0   2024        1    ABE  ABQ         1.0 2024-02-10        0         0   
1   2024        1    ABE  ABQ         1.0 2024-01-31        0         0   
2   2024        1    ABE  ABQ         1.0 2024-03-21        0         0   
3   2024        1    ABE  ABQ         1.0 2024-02-24        0         0   
4   2024        1    ABE  ABQ         1.0 2024-01-12        0         0   
5   2024        1    ABE  ABQ         1.0 2024-01-12        0         0   
6   2024        1    ABE  ABQ         1.0 2024-02-07        0         0   
7   2024        1    ABE  ABQ         1.0 2024-01-09        0         0   
8   2024        1    ABE  ABQ         1.0 2024-03-08        0         0   
9   2024        1    ABE  ABQ         1.0 2024-03-18        0         0   
10  2024        1    ABE  ABQ         1.0 2024-03-01        0         0   
11  2024        1    ABE  ABQ         1.0 2024-03-16        0         0   
12  2024        1    ABE 

In [ ]:
#Guardar archivo con atributos necesarios
df_data_import['PASSENGERS']=df_data_import['PASSENGERS'].astype(int)
print(df_data_import.head(50))

df_data_import.to_csv('dataset_fligth_predict.csv',index=False)

    YEAR  QUARTER ORIGIN DEST  PASSENGERS       DATE  HOLIDAY  VACATION  \
0   2024        1    ABE  ABQ           1 2024-02-10        0         0   
1   2024        1    ABE  ABQ           1 2024-01-31        0         0   
2   2024        1    ABE  ABQ           1 2024-03-21        0         0   
3   2024        1    ABE  ABQ           1 2024-02-24        0         0   
4   2024        1    ABE  ABQ           1 2024-01-12        0         0   
5   2024        1    ABE  ABQ           1 2024-01-12        0         0   
6   2024        1    ABE  ABQ           1 2024-02-07        0         0   
7   2024        1    ABE  ABQ           1 2024-01-09        0         0   
8   2024        1    ABE  ABQ           1 2024-03-08        0         0   
9   2024        1    ABE  ABQ           1 2024-03-18        0         0   
10  2024        1    ABE  ABQ           1 2024-03-01        0         0   
11  2024        1    ABE  ABQ           1 2024-03-16        0         0   
12  2024        1    ABE 

Cargar y Explorar los Datos

In [ ]:
#Training

#Load data
df_dataset=pd.read_csv('dataset_fligth_predict.csv')
df_features=df_dataset[['ORIGIN','DEST','DATE','HOLIDAY','VACATION','IMPORTANT_EVENT','PASSENGERS']].copy()

le_dest=LabelEncoder()
df_features['DEST_ENCONDED']=le_dest.fit_transform(df_features['DEST'])

le_origin=LabelEncoder()
df_features['ORIGIN_ENCONDED']=le_origin.fit_transform(df_features['ORIGIN'])

df_features['DATE']=pd.to_datetime(df_features['DATE'])

#extract features time
df_features['MONTH']=df_features['DATE'].dt.month
df_features['DAY_WEEK']=df_features['DATE'].dt.day_of_week

# Definir variable objetivo (destinos populares = top 10 con más pasajeros)
top_destinos = df_features.groupby('DEST')['PASSENGERS'].sum().nlargest(10).index
df_features['IS_POPULAR'] = df['DEST'].isin(top_destinos).astype(int)

print(df_features.head(10))



  ORIGIN DEST       DATE  HOLIDAY  VACATION  IMPORTANT_EVENT  PASSENGERS  \
0    ABE  ABQ 2024-02-10        0         0                0           1   
1    ABE  ABQ 2024-01-31        0         0                0           1   
2    ABE  ABQ 2024-03-21        0         0                0           1   
3    ABE  ABQ 2024-02-24        0         0                0           1   
4    ABE  ABQ 2024-01-12        0         0                0           1   
5    ABE  ABQ 2024-01-12        0         0                0           1   
6    ABE  ABQ 2024-02-07        0         0                0           1   
7    ABE  ABQ 2024-01-09        0         0                0           1   
8    ABE  ABQ 2024-03-08        0         0                0           1   
9    ABE  ABQ 2024-03-18        0         0                0           1   

   DEST_ENCONDED  ORIGIN_ENCONDED  MONTH  DAY_WEEK  IS_POPULAR  
0              2                0      2         5           0  
1              2                0

In [ ]:
#Guardados de los encodes
df_features[['DEST','DEST_ENCONDED']].drop_duplicates().to_csv('dest_airport_encoded.csv',index=False)
df_features[['ORIGIN','ORIGIN_ENCONDED']].drop_duplicates().to_csv('origin_airport_encoded.csv',index=False)

Dividir Datos en Train/Test

In [53]:
features=['MONTH','DAY_WEEK','HOLIDAY','VACATION','IMPORTANT_EVENT','ORIGIN_ENCONDED','DEST_ENCONDED']

X=df_features[features]

y=df_features['IS_POPULAR']
print(X.head(10),y.head(10))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



   MONTH  DAY_WEEK  HOLIDAY  VACATION  IMPORTANT_EVENT  ORIGIN_ENCONDED  \
0      2         5        0         0                0                0   
1      1         2        0         0                0                0   
2      3         3        0         0                0                0   
3      2         5        0         0                0                0   
4      1         4        0         0                0                0   
5      1         4        0         0                0                0   
6      2         2        0         0                0                0   
7      1         1        0         0                0                0   
8      3         4        0         0                0                0   
9      3         0        0         0                0                0   

   DEST_ENCONDED  
0              2  
1              2  
2              2  
3              2  
4              2  
5              2  
6              2  
7              2  
8  

Normalización de Datos

In [54]:
# Normalizar datos numéricos (excepto variables binarias)
scaler = StandardScaler()
numeric_cols = ['ORIGIN_ENCONDED', 'DEST_ENCONDED', 'DAY_WEEK','MONTH']
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [ ]:
#guadado del scalado (normalizado)
joblib.dump(scaler,'scaler.pkl')



['scaler.pkl']

Entrenar el Modelo

In [55]:
#Construcción del Modelo en TensorFlow/Keras

from keras.models import Sequential
from keras.layers import Dense,Dropout,Input,BatchNormalization
from keras.optimizers import Adam


model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')  # clasificación binaria
])

print(X_train.shape[1])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall']
              )

model.summary()

7


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,857 (38.50 KB)

 Trainable params: 9,601 (37.50 KB)

 Non-trainable params: 256 (1.00 KB)

In [ ]:

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
#Entrenamiento
history=model.fit(X_train,y_train,epochs=10,batch_size=1024,validation_split=0.2,callbacks=[early_stopping],class_weight={0:1,1:10})

Epoch 1/10
4632/4632 ━━━━━━━━━━━━━━━━━━━━ 32s 6ms/step - Precision: 0.3264 - Recall: 0.9878 - accuracy: 0.5007 - loss: 1.3753 - val_Precision: 0.4571 - val_Recall: 1.0000 - val_accuracy: 0.7117 - val_loss: 0.6140
Epoch 2/10
4632/4632 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - Precision: 0.4687 - Recall: 0.9958 - accuracy: 0.7243 - loss: 0.8947 - val_Precision: 0.5178 - val_Recall: 1.0000 - val_accuracy: 0.7740 - val_loss: 0.4331
Epoch 3/10
4632/4632 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - Precision: 0.5689 - Recall: 0.9925 - accuracy: 0.8153 - loss: 0.6313 - val_Precision: 0.6294 - val_Recall: 1.0000 - val_accuracy: 0.8571 - val_loss: 0.3373
Epoch 4/10
4632/4632 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - Precision: 0.6264 - Recall: 0.9912 - accuracy: 0.8545 - loss: 0.5288 - val_Precision: 0.6424 - val_Recall: 1.0000 - val_accuracy: 0.8649 - val_loss: 0.3166
Epoch 5/10
4632/4632 ━━━━━━━━━━━━━━━━━━━━ 28s 6ms/step - Precision: 0.6538 - Recall: 0.9919 - accuracy: 0.8706 - loss: 0.4698 - val_Precision: 0.693

Evaluación del Modelo

In [58]:
#Evaluacion
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]:.4f}, Test Accuracy: {results[1]:.4f}")

# Predecir destinos populares
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Umbral de 0.5

# Matriz de confusión
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

46313/46313 ━━━━━━━━━━━━━━━━━━━━ 125s 3ms/step - Precision: 0.8407 - Recall: 0.9363 - accuracy: 0.9413 - loss: 0.1670
Test Loss: 0.1667, Test Accuracy: 0.9415
46313/46313 ━━━━━━━━━━━━━━━━━━━━ 30s 645us/step
[[1057253   64054]
 [  22693  337987]]


Guardar el Modelo y Componentes

In [ ]:
model.save('modelo_destinos_populares.keras')

Pruebas del modelo guardado

In [ ]:

model_load = load_model('model_dest_popular.keras',compile=False)
# Generar un input aleatorio respetando los rangos de las variables
random_input = np.array([[
    np.random.choice([1, 2, 3]),  # MONTH (puede ser 1, 2 o 3)
    np.random.randint(0, 7),      # DAY_WEEK (0-6)
    np.random.randint(0, 2),      # HOLIDAY (0 o 1)
    np.random.randint(0, 2),      # VACATION (0 o 1)
    np.random.randint(0, 2),      # IMPORTANT_EVENT (0 o 1)
    int(np.random.uniform(X['ORIGIN_ENCONDED'].min(), X['ORIGIN_ENCONDED'].max())),  # ORIGIN_ENCONDED
    int(np.random.uniform(X['DEST_ENCONDED'].min(), X['DEST_ENCONDED'].max()))       # DEST_ENCONDED
]])
print(random_input)
# Normalizar las columnas numéricas igual que en el entrenamiento
random_input_df = pd.DataFrame(random_input, columns=features)

random_input_df[numeric_cols] = scaler.transform(random_input_df[numeric_cols])
print(random_input_df)
# Predicción
pred = model_load.predict(random_input_df)


print(f"Probabilidad de destino popular: {pred[0][0]:.4f}")
print("Predicción binaria:", int(pred[0][0] > 0.5))

[[  3   1   0   1   1 142 226]]
      MONTH  DAY_WEEK  HOLIDAY  VACATION  IMPORTANT_EVENT  ORIGIN_ENCONDED  \
0  1.211967 -1.000254        0         1                1         -0.70129   

   DEST_ENCONDED  
0       0.012189  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Probabilidad de destino popular: 0.9961
Predicción binaria: 1


0
